In [1]:
try:
  flag = dbutils.fs.mount(
  source = "wasbs://bigdataprojectblob@bigdatasa001.blob.core.windows.net",
 mount_point = "/mnt/get",
 extra_configs = {"fs.azure.account.key.bigdatasa001.blob.core.windows.net":"AiXXEvh75+sny+87xUATW2eXQjllE6zyh4BBEPXek7Y5hs9NaxjBLhroktKmLzH6L/1foLO1NXXnSFtUZJfkLQ=="})
except:
  print("Exception While mounting")

In [2]:
dbutils.fs.mkdirs('/FileStore/tables/csv_files')
dbutils.fs.mkdirs('FileStore/tables/json_files')

Out[65]: True

In [3]:
list_of_files = dbutils.fs.ls('/mnt/get/')
for file in list_of_files:
  if(file.name.endswith('json')):
    dbutils.fs.cp('/mnt/get/'+file.name, '/FileStore/tables/json_files/')
  if(file.name.endswith('csv')):
    dbutils.fs.cp('/mnt/get/'+file.name, '/FileStore/tables/csv_files/')
dbutils.fs.ls('/mnt/get/')[0]

Out[66]: FileInfo(path='dbfs:/mnt/get/CA_category_id.json', name='CA_category_id.json', size=7911)

In [4]:
dbutils.fs.unmount("/mnt/get")

/mnt/get has been unmounted.
Out[67]: True

In [5]:
country_list = [fileinfo.name[0:2] for fileinfo in dbutils.fs.ls('/FileStore/tables/csv_files')]

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from  pyspark.sql.functions import *
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

In [7]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Youtube Data") \
    .config("spark.youtube.config.option", "youtube-eda") \
    .getOrCreate()

In [8]:
schema = StructType([
   StructField("video_id", StringType(), True),
   StructField("trending_date", StringType(), True),
   StructField("title", StringType(), True),
   StructField("channel_title", StringType(), True),
   StructField("category_id", StringType(), True),
   StructField("publish_time", TimestampType(), True),
   StructField("tags", StringType(), True),
   StructField("views", IntegerType(), True),
   StructField("likes", IntegerType(), True),
   StructField("dislikes", IntegerType(), True),
   StructField("comment_count", IntegerType(), True),
   StructField("thumbnail_link", StringType(), True),
   StructField("comments_disabled", BooleanType(), True),
   StructField("ratings_disabled", BooleanType(), True),
   StructField("video_error_or_removed", BooleanType(), True),
   StructField("description", StringType(), True),
])

In [9]:
country_csv = list()
for countrycode in country_list:
  youtube_csv = spark.read.csv('dbfs:/FileStore/tables/csv_files/'+countrycode+'videos.csv', header=True, schema=schema).withColumn('country', lit(countrycode))
  country_csv.append(youtube_csv)

In [10]:
youtube_df = reduce(DataFrame.unionAll,country_csv)

In [11]:
youtube_df.count()

Out[197]: 416869

In [12]:
youtube_df.show(5)

+-----------+-------------+--------------------+-------------+-----------+-------------------+--------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-------+
 video_id|trending_date| title|channel_title|category_id| publish_time| tags| views| likes|dislikes|comment_count| thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed| description|country|
+-----------+-------------+--------------------+-------------+-----------+-------------------+--------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-------+
n1WpP7iowLc| 17.14.11|Eminem - Walk On ...| EminemVEVO| 10|2017-11-10 17:00:03|"Eminem"|"Walk"|"...|17158579| 787425| 43420| 125882|https://i.ytimg.c...| false| false| false|Eminem's new trac...| CA|
0dBIkQ4Mz1M| 17.14.11|PLUSH - Bad Unbox...| iDubbbzTV| 23|2017-11-13 17:00:00|"plush"|"bad unbo...| 1014651| 127794| 1688| 13030|https://i.ytimg.c...| false| false| false|STill got a lot o...| CA|
5qpjK5DgCt4| 17.14.11|Racist Superman |...| Rudy Mancuso| 23|2017-11-12 19:05:24|"racist superman"...| 3191434| 146035| 5339| 8181|https://i.ytimg.c...| false| false| false|WATCH MY PREVIOUS...| CA|
d380meD0W0M| 17.14.11|I Dare You: GOING...| nigahiga| 24|2017-11-12 18:01:41|"ryan"|"higa"|"hi...| 2095828| 132239| 1989| 17518|https://i.ytimg.c...| false| false| false|I know it's been ...| CA|
2Vv-BfVoq4g| 17.14.11|Ed Sheeran - Perf...| Ed Sheeran| 10|2017-11-09 11:04:14|"edsheeran"|"ed s...|33523622|1634130| 21082| 85067|https://i.ytimg.c...| false| false| false|🎧: https://ad.gt...| CA|
+-----------+-------------+--------------------+-------------+-----------+-------------------+--------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-------+
only showing top 5 rows

In [13]:
youtube.select('trending_date', 'publish_time').show(5, False)

+-------------+-------------------+
trending_date|publish_time |
+-------------+-------------------+
17.14.11 |2017-11-13 09:09:31|
17.14.11 |2017-11-13 17:32:11|
17.14.11 |2017-11-13 16:11:31|
17.14.11 |2017-11-13 06:51:10|
17.14.11 |2017-11-13 16:52:36|
+-------------+-------------------+
only showing top 5 rows

In [14]:
ca_categories = spark.read.option('multiline', True).json('/FileStore/tables/json_files/CA_category_id.json')

In [15]:
schema_json = StructType([
   StructField("etag", StringType(), True),
   StructField("kind", StringType(), True),
   StructField("items", ArrayType(StructType([
                         StructField('kind', StringType(), True),
                         StructField('etag', StringType(), True),
                         StructField('id', StringType(), True),
                         StructField('snippet', StructType([
                                     StructField('channelId', StringType(),True),
                                     StructField('title', StringType(), True),
                                     StructField('assignable', BooleanType(), True)])
                                                                                     , True)])
               , True),True)
])

In [17]:
category_df_list = list()
for countrycode in country_list:
  ca_categories = spark.read.option('multiline', True).json('dbfs:/FileStore/tables/json_files/'+countrycode+'_category_id.json', schema=schema_json)
  items_df = ca_categories.select(explode('items'))
  temp = items_df.select(col('col').getItem('id').alias('id'), col('col').getItem('snippet').getItem('channelId').alias('channelId'), col('col').getItem('snippet').getItem('title').alias('title')).withColumn('country', lit(countrycode))
  category_df_list.append(temp)

In [18]:
category_d= reduce(DataFrame.unionAll, category_df_list)

In [19]:
category_d.columns

Out[209]: ['id', 'channelId', 'title', 'country']

In [20]:
category_d.show(5)

+---+--------------------+----------------+-------+
 id| channelId| title|country|
+---+--------------------+----------------+-------+
 1|UCBR8-60-B28hp2Bm...|Film & Animation| CA|
 2|UCBR8-60-B28hp2Bm...|Autos & Vehicles| CA|
 10|UCBR8-60-B28hp2Bm...| Music| CA|
 15|UCBR8-60-B28hp2Bm...| Pets & Animals| CA|
 17|UCBR8-60-B28hp2Bm...| Sports| CA|
+---+--------------------+----------------+-------+
only showing top 5 rows

In [21]:
category_d.select('title').distinct().count()

Out[215]: 301